# Estudios de Casos de Modelos Analíticos I

<br>**Contenido:**<br>
**1-** Presentación de la librería yfinance<br>
**2-** Presentación de la librería pandas-datareader<br>
**3-** Predicciones utilizando Scikit-Learn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns
import pandas_datareader.data as web
import datetime

# 1- yfinance
**Instalación para Jupyter:**
    pip install yfinance<br>
**Instalación para Colab:** !pip install yfinance<br>
**Documentación:**
    https://pypi.org/project/yfinance/

In [ ]:
BMA_yf = yf.Ticker("BMA.BA")
GGAL_yf = yf.Ticker("GGAL.BA")
BPAT_yf = yf.Ticker("BPAT.BA")
BBAR_yf = yf.Ticker("BBAR.BA")
BRIO_yf = yf.Ticker("BRIO.BA")
empresas = [BMA_yf, GGAL_yf, BPAT_yf, BBAR_yf, BRIO_yf]

In [ ]:
BMA_yf.info

In [ ]:
def convertir_DataFrame(empresas):
    marketCap_values = []
    empleados_values = []
    cierre_values = []
    EPS_values = []
    index_df = []
    for item in empresas:
        marketCap_values.append(item.info["marketCap"] / 1000000000)
        empleados_values.append(item.info["fullTimeEmployees"])
        cierre_values.append(item.info["previousClose"])
        EPS_values.append(item.info["trailingEps"])
        symbol = item.info["symbol"]
        index_df.append(symbol)

    data_df = {
        "Capitalización de Mercado": marketCap_values,
        "Cantidad de Empleados": empleados_values,
        "Último Cierre": cierre_values,
        "Ganancias por acción": EPS_values,
    }
    full_df = pd.DataFrame(data=data_df, index=index_df)
    return full_df

In [ ]:
empresas_df = convertir_DataFrame(empresas)
empresas_df.head()

In [ ]:
barras = sns.barplot(
    data=empresas_df, x=empresas_df.index, y=empresas_df["Capitalización de Mercado"]
)
barras.axhline(empresas_df["Capitalización de Mercado"].mean(), color="purple")
plt.show()

In [ ]:
barras = sns.barplot(
    data=empresas_df, x=empresas_df.index, y=empresas_df["Cantidad de Empleados"]
)
barras.axhline(empresas_df["Cantidad de Empleados"].mean(), color="purple")
plt.show()

In [ ]:
scatter = sns.scatterplot(
    data=empresas_df,
    x=empresas_df["Último Cierre"],
    y=empresas_df["Ganancias por acción"],
    s=empresas_df["Capitalización de Mercado"],
    hue=empresas_df.index,
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.show()

# 2- pandas-datareader
**Instalación para Jupyter:**
    pip install pandas-datareader<br>
**Instalación para Colab:** !pip install --upgrade pandas_datareader<br>
En caso de no funcionar, volver a ejecutar tras haber reiniciado el entorno de ejecución.
**Documentación:**
    https://pandas-datareader.readthedocs.io/en/latest/